In [1]:
import pandas as pd
import numpy as np
from category_encoders import BinaryEncoder
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import MultiLabelBinarizer
from sklearn.preprocessing import OneHotEncoder
from sklearn.model_selection import train_test_split
import random
from datetime import datetime
import pgeocode
import cupy as cp

pd.options.display.max_columns = None
pd.options.display.max_rows = None

In [2]:
p = 0.1
df = pd.read_csv('real_estate_raw.csv')
             #    skiprows=lambda i: i>0 and random.random() > p)

/tmp/ipykernel_4272/3526927334.py:2: DtypeWarning: Columns (16,134) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv('real_estate_raw.csv')


In [3]:
cols = [
    'original_list_price',
    'list_price',
    'close_price',
    'association_fee',
    'tax_annual_amount',
    'days_on_market',
    'cumulative_days_on_market',
    'previous_list_price',
    'living_area',
    'lot_size_acres',

    'rooms_total',
    'bedrooms_total',
    'bathrooms_full',
    'bathrooms_half',
    'garage_spaces',
 
    'year_built',
    
    'postal_code',

    'elementary_school_district',
    'middle_or_junior_school_district',
    'high_school_district',
    
    'accessibility_features',
    'heating',
    'water_source',
    'sewer',
    'lot_features',
    'roof',
    'community_features',
    'laundry_features',
    'cooling',
    'association_fee_includes',
    'mrd_din',
    'mrd_ext',
    'mrd_fireplace_location',
    'ownership',
    'mrd_bas',
    'mrd_pkn',
    
    'waterfront_yn',
    'mrd_disability_access',
    'mrd_garage_onsite',
    'new_construction_yn',
        
    'mrd_rehab_year',
        
    'mrd_tnu',
    'mrd_tpc',
    'mrd_tpe'
]

In [4]:
df = df[cols]

In [5]:
df.info(verbose=True,show_counts=True)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 400918 entries, 0 to 400917
Data columns (total 44 columns):
 #   Column                            Non-Null Count   Dtype  
---  ------                            --------------   -----  
 0   original_list_price               400918 non-null  float64
 1   list_price                        400918 non-null  float64
 2   close_price                       400918 non-null  float64
 3   association_fee                   400918 non-null  float64
 4   tax_annual_amount                 400918 non-null  float64
 5   days_on_market                    400918 non-null  int64  
 6   cumulative_days_on_market         400918 non-null  int64  
 7   previous_list_price               400918 non-null  int64  
 8   living_area                       400918 non-null  float64
 9   lot_size_acres                    400918 non-null  float64
 10  rooms_total                       400918 non-null  int64  
 11  bedrooms_total                    400918 non-null  i

In [6]:
class DataPreprocess:
    def __init__(self):
        self.all_schools_dict = {}
        self.ohe_encoder_dict = {}
        self.school_binarizer = {}
        self.feature_binarizer = {}
        self.numeric_value_normalizer = None
        self.numeric_target_normalizer = None
        self.feature_list =['accessibility_features',
                                'heating',
                                'water_source',
                                'sewer',
                                'lot_features',
                                'roof',
                                'community_features',
                                'laundry_features',
                                'cooling',
                                'association_fee_includes',
                                'mrd_din',
                                'mrd_ext',
                                'mrd_fireplace_location',
                                'ownership',
                                'mrd_bas',
                                'mrd_pkn',
                                'mrd_tpc',
                                'mrd_tpe']
        self.school_features = ['elementary_school_district','middle_or_junior_school_district','high_school_district']
        self.room_features = ['rooms_total','bedrooms_total','bathrooms_full','bathrooms_half','garage_spaces']
        self.all_multi_binarized_features = []
        self.all_postal_binarized_features = []
        self.all_boolean_features = []
        self.all_continuous_features = ['original_list_price',
                                        'list_price',
                                        'association_fee',
                                        'tax_annual_amount',
                                        'days_on_market',
                                        'cumulative_days_on_market',
                                        'previous_list_price',
                                        'living_area',
                                        'lot_size_acres']
        

    def clean_schools(self, df, dataset='train', schools=None):
        ## find missing school districts by zip code ##
        if schools is None:
            schools = self.school_features
        
        if dataset == 'train':
            before = df.elementary_school_district.isna().sum(),df.middle_or_junior_school_district.isna().sum(),df.high_school_district.isna().sum()
            
            for dist in schools:
                df[dist] = df[dist].apply(lambda x: str(x).lower() if pd.notnull(x) else x)
                
            for dist in schools:
                df_school = df[['postal_code',dist]].copy()
                df_school = df_school.dropna(subset=[dist],how='all')
                
                df_school = df_school.groupby(["postal_code",dist]).size().reset_index().rename(columns={0:'count'})
                school_dict = dict(zip(df_school.postal_code, df_school[dist]))
                self.all_schools_dict[dist] = school_dict
            
                df[dist] = df[dist].fillna(df.postal_code.map(school_dict))
            
            after = df.elementary_school_district.isna().sum(),df.middle_or_junior_school_district.isna().sum(),df.high_school_district.isna().sum()
            print(f'Reduced district nulls from {before} to {after} after.')
            df[schools] = df[schools].fillna(0)
            
        elif dataset == 'predict':
            before = df.elementary_school_district.isna().sum(),df.middle_or_junior_school_district.isna().sum(),df.high_school_district.isna().sum()
            for dist in schools:
                df[dist] = df[dist].apply(lambda x: str(x).lower() if pd.notnull(x) else x)
                
            for dist in schools:
                school_dict = self.all_schools_dict[dist]
                df[dist] = df[dist].fillna(df.postal_code.map(school_dict))

            after = df.elementary_school_district.isna().sum(),df.middle_or_junior_school_district.isna().sum(),df.high_school_district.isna().sum()
            print(f'Reduced district nulls from {before} to {after} after.') 
            
            df[schools] = df[schools].fillna(0)

        return df
            
    def binarize_schools(self, df, dataset='train', schools=None):   
        ## binarize school districts transform ##
        if schools is None:
            schools = self.school_features

        binarized_features = []
        
        if dataset == 'train':
            
            for dist in schools:
                df_dist = df[dist].copy()
                encoder = BinaryEncoder(cols=[dist])
                df_binarize = encoder.fit_transform(df_dist)
                df = pd.concat([df,df_binarize],axis=1)
                df = df.drop(dist,axis=1)
                binarized_features.extend(encoder.feature_names_out_)
                self.school_binarizer[dist] = encoder
                
        elif dataset == 'predict':
            
            for dist in schools:
                df_dist = df[dist].copy()
                df_binarize = self.school_binarizer[dist].transform(df_dist)
                df = pd.concat([df,df_binarize],axis=1)
                df = df.drop(dist,axis=1)

        self.all_postal_binarized_features.extend(binarized_features)
        df[binarized_features] = df[binarized_features].astype('bool')
        
        return df 

    def clean_age(self, df):
        ## building age transform ##
        current_year = datetime.now().year
        
        df['age'] = df['year_built'].apply(lambda x: current_year-int(x) if int(x) > 0 else x)
        bins = [-1, 0, 10, 20, 30, 40, 50, 60, 70, 80, 1000]
        labels = ['0','1-10', '10-20', '20-30', '30-40', '40-50', '50-60', '60-70', '70-80', '80+']
        df['age_label'] = pd.cut(df.age, bins=bins, labels=labels)
        df = df.drop('age',axis=1)

        df["age_label"] = df["age_label"].astype("category")
        
        return df

    def clean_booleans(self, df):
        ## boolean fields transform ##
        df['waterfront_yn'] = df['waterfront_yn'].fillna(False)
        df['new_construction_yn'] = df['new_construction_yn'].fillna(False)
        df['mrd_disability_access'] = df['mrd_disability_access'].fillna('No')
        df['mrd_disability_access'] = pd.Series(np.where(df.mrd_disability_access.values == 'Yes', True, False), df.index)
        df['mrd_garage_onsite'] = df['mrd_garage_onsite'].fillna('No')
        df['mrd_garage_onsite'] = pd.Series(np.where(df.mrd_garage_onsite.values == 'Yes', True, False), df.index)
        df['mrd_rehab_year'] = df['mrd_rehab_year'].apply(lambda x: True if pd.notnull(x) else False)

        boolean = ['waterfront_yn','new_construction_yn','mrd_disability_access','mrd_garage_onsite','mrd_rehab_year']
        df[boolean] = df[boolean].astype('bool')

        return df

    def clean_room_features(self, df, room_list=None):
        ## rooms binning transform ##
        if room_list is None:
            room_list = self.room_features
        
        for feature in room_list:
            min_val = -1
            zero_val = 0
            first_quant = 1
            second_quant = 2
            third_quant = 3
            last_quant = 4
            extra_quant = 5
            max_val = 1000
            bins = [min_val,zero_val,first_quant,second_quant,third_quant,last_quant,extra_quant,max_val]
            labels = [f'{zero_val}',f'{first_quant}',f'{second_quant}',f'{third_quant}',f'{last_quant}',f'{extra_quant}',f'{extra_quant}+']
            df[feature] = pd.cut(df[feature], bins=bins, labels=labels, duplicates='drop')

        df[room_list] = df[room_list].astype('category')
            
        return df

    def clean_postal_codes(self, df):
        ## postal code to long lat coordinates ##
        geolocator = pgeocode.Nominatim('US')
        geocode_dict = {}
        for postal_code in list(df.postal_code.unique()):
            location = geolocator.query_postal_code(str(postal_code))
            geocode_dict[postal_code] = (location.longitude,location.latitude)
        
        df['postal_long'] = df['postal_code'].apply(lambda x: geocode_dict[x][0] if ((pd.notnull(x)) and (x in geocode_dict)) else np.nan)
        df['postal_lat'] = df['postal_code'].apply(lambda x: geocode_dict[x][1] if ((pd.notnull(x)) and (x in geocode_dict)) else np.nan)
        
        df.postal_long = df.postal_long.fillna(0)
        df.postal_lat = df.postal_lat.fillna(0)

        df[['postal_long','postal_lat']] = df[['postal_long','postal_lat']].astype('float32')
        add_list = ['postal_long','postal_lat']
        for el in add_list:
            if el not in self.all_continuous_features:
                self.all_continuous_features.extend(['postal_long','postal_lat'])

        return df

    def clean_multi_label_features(self, df, feature_list=None):
        ## splitting of features into individual lists per cell for labelizer ##
        if feature_list is None:
            feature_list = self.feature_list
            
        for feature in feature_list:
            df[feature] = df[feature].fillna(f'None_{feature}')
            curr_len=(len(df[feature].str.split(',\s*').explode().unique().tolist()))
            print(f'{feature} has {curr_len} unique categories.')
            df[feature] = df[feature].str.replace('[{}"]','',regex=True).str.split(',\s*')

        return df

    def multi_label_binarizer(self, df, dataset='train', feature_list=None):
        
        if feature_list is None:
            feature_list = self.feature_list

        if dataset == 'train':
            for feature in feature_list:
                mlb = MultiLabelBinarizer()
                df = df.join(
                        pd.DataFrame(
                                mlb.fit_transform(df.pop(feature)),
                                index=df.index,
                                columns=mlb.classes_), lsuffix=f'{feature}_')
                self.feature_binarizer[feature] = mlb
                self.all_multi_binarized_features.extend(list(mlb.classes_))
                
        elif dataset == 'predict':
            for feature in feature_list:
                mlb = self.feature_binarizer[feature]
                df = df.join(
                        pd.DataFrame(
                                mlb.transform(df.pop(feature)),
                                index=df.index,
                                columns=mlb.classes_), lsuffix=f'{feature}_')

        

        for col in self.all_multi_binarized_features:
            df[col] = df[col].astype('bool')
        
        return df

    def normalize_numeric(self, df, dataset='train', numerical = None):
        ## taking numeric continous features and normalizing them using a min max scaler ##
        if numerical is None:
            numerical = self.all_continuous_features
            
        if dataset == 'train':
            print(df[numerical].columns)
            scaler = MinMaxScaler()
            df[numerical] = scaler.fit_transform(df[numerical])
            self.numeric_value_normalizer = scaler
            
        elif dataset == 'predict':
            print(df[numerical].columns)
            scaler = self.numeric_value_normalizer
            df[numerical] = scaler.transform(df[numerical])

        df[numerical] = df[numerical].astype('float32')
            
        return df

    def normalize_target(self, df, dataset='train',target=None):
        ## taking the target feature and normalizing it using a min max scaler ##
        if target is None:
            target = 'close_price'
            
        if dataset == 'train':
            target_scaler = MinMaxScaler()
            df[target] = target_scaler.fit_transform(df[target].to_numpy().reshape(-1, 1))
            self.numeric_target_normalizer = target_scaler
        elif dataset == 'predict':
            target_scaler = self.numeric_target_normalizer
            df[target] = target_scaler.transform(df[target].to_numpy().reshape(-1, 1))

        df[target] = df[target].astype('float32')

        return df

    def ohe_categories(self, df, dataset='train', categorical=None):

        if categorical is None:
            categorical = list(df.select_dtypes(include=['category']).columns)
    
        if dataset == 'train':
        
            for cat in categorical:
                df_ohe = df[cat].values.reshape(-1, 1)
                enc = OneHotEncoder(handle_unknown = "ignore").fit(df_ohe)
                df_ohe = enc.transform(df_ohe).toarray()
                df_ohe = pd.DataFrame(df_ohe, columns=enc.categories_)
                df_ohe = df_ohe.add_prefix(f'{cat}_')
                self.all_boolean_features.extend(list(df_ohe.columns))
                df = pd.concat([df,df_ohe], axis=1)
                df = df.drop(cat,axis=1)
                self.ohe_encoder_dict[cat] = enc

        elif dataset == 'predict':
        
            for cat in categorical:
                df_ohe = df[cat].values.reshape(-1, 1)
                enc = self.ohe_encoder_dict[cat]
                df_ohe = enc.transform(df_ohe).toarray()
                df_ohe = pd.DataFrame(df_ohe, columns=enc.categories_)
                df_ohe = df_ohe.add_prefix(f'{cat}_')
                df = pd.concat([df,df_ohe], axis=1)
                df = df.drop(cat,axis=1)
                self.ohe_encoder_dict['cat'] = enc

        for col in self.all_boolean_features:
            df[col] = df[col].astype('bool')
        
        return df
            
        

In [7]:
y = df[['close_price']]
X = df.drop('close_price',axis=1)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=42)
X_train = X_train.reset_index(drop=True)
X_test = X_test.reset_index(drop=True)
y_train = y_train.reset_index(drop=True)
y_test = y_test.reset_index(drop=True)

In [8]:
dp = DataPreprocess()

def preprocess_train(df):
    df = dp.clean_schools(df,dataset='train')
    df = dp.binarize_schools(df,dataset='train')
    df = dp.clean_age(df)
    df = dp.clean_booleans(df)
    df = dp.clean_room_features(df)
    df = dp.clean_postal_codes(df)
    df = dp.clean_multi_label_features(df)
    df = dp.multi_label_binarizer(df,dataset='train')
    df = dp.normalize_numeric(df,dataset='train')
    df = dp.ohe_categories(df,dataset='train')
    return df
X_train = preprocess_train(X_train)

Reduced district nulls from (1428, 1451, 1514) to (476, 488, 481) after.
accessibility_features has 222 unique categories.
heating has 83 unique categories.
water_source has 26 unique categories.
sewer has 28 unique categories.
lot_features has 211 unique categories.
roof has 33 unique categories.
community_features has 66 unique categories.
laundry_features has 45 unique categories.
cooling has 57 unique categories.
association_fee_includes has 85 unique categories.
mrd_din has 7 unique categories.
mrd_ext has 35 unique categories.
mrd_fireplace_location has 16 unique categories.
ownership has 6 unique categories.
mrd_bas has 25 unique categories.
mrd_pkn has 4 unique categories.
mrd_tpc has 30 unique categories.
mrd_tpe has 18 unique categories.
Index(['original_list_price', 'list_price', 'association_fee',
       'tax_annual_amount', 'days_on_market', 'cumulative_days_on_market',
       'previous_list_price', 'living_area', 'lot_size_acres', 'postal_long',
       'postal_lat'],
    

In [9]:
def preprocess_target_train(df):
    df = dp.normalize_target(df,dataset='train')
    return df
    
y_train = preprocess_target_train(y_train)

In [10]:
y_train.head()

,close_price
0,0.007833
1,0.003194
2,0.002083
3,0.010072
4,0.005000


In [11]:
def preprocess_test(df):
    df = dp.clean_schools(df,dataset='predict')
    df = dp.binarize_schools(df,dataset='predict')
    df = dp.clean_age(df)
    df = dp.clean_booleans(df)
    df = dp.clean_room_features(df)
    df = dp.clean_postal_codes(df)
    df = dp.clean_multi_label_features(df)
    df = dp.multi_label_binarizer(df,dataset='predict')
    df = dp.normalize_numeric(df,dataset='predict')
    df = dp.ohe_categories(df,dataset='predict')
    return df

X_test = preprocess_test(X_test)

Reduced district nulls from (703, 715, 741) to (247, 250, 258) after.
accessibility_features has 208 unique categories.
heating has 82 unique categories.
water_source has 26 unique categories.
sewer has 27 unique categories.
lot_features has 211 unique categories.
roof has 31 unique categories.
community_features has 65 unique categories.
laundry_features has 45 unique categories.
cooling has 52 unique categories.
association_fee_includes has 84 unique categories.
mrd_din has 7 unique categories.
mrd_ext has 34 unique categories.
mrd_fireplace_location has 16 unique categories.
ownership has 6 unique categories.
mrd_bas has 25 unique categories.
mrd_pkn has 4 unique categories.
mrd_tpc has 30 unique categories.
mrd_tpe has 18 unique categories.


/home/charon/.local/lib/python3.10/site-packages/sklearn/preprocessing/_label.py:900: UserWarning: unknown class(es) ['Coal'] will be ignored
  warnings.warn(


Index(['original_list_price', 'list_price', 'association_fee',
       'tax_annual_amount', 'days_on_market', 'cumulative_days_on_market',
       'previous_list_price', 'living_area', 'lot_size_acres', 'postal_long',
       'postal_lat'],
      dtype='object')


In [12]:
def preprocess_target_test(df):
    df = dp.normalize_target(df,dataset='predict')
    return df
    
y_test = preprocess_target_test(y_test)

In [13]:
X_train = X_train.drop(['postal_code'],axis=1)
X_test = X_test.drop(['postal_code'],axis=1)

X_train.mrd_tnu = X_train.mrd_tnu.fillna(0)
X_test.mrd_tnu = X_test.mrd_tnu.fillna(0)

In [32]:
import xgboost as xgb
from sklearn.metrics import auc, accuracy_score, confusion_matrix, mean_squared_error
from sklearn.model_selection import cross_val_score, GridSearchCV, KFold, RandomizedSearchCV, train_test_split

#xgb_model = xgb.XGBRegressor(objective="reg:squarederror", n_estimators=80,learning_rate=0.11669835581158701, max_depth=6, subsample=0.6628468243767216)
xgb_model = xgb.XGBRegressor(objective="reg:squarederror", n_estimators=80)

# X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42)

xgb_model.fit(X_train, y_train, eval_set=[(X_test, y_test)])

y_pred = xgb_model.predict(X_test)

[0]	validation_0-rmse:0.00675
[1]	validation_0-rmse:0.00508
[2]	validation_0-rmse:0.00398
[3]	validation_0-rmse:0.00331
[4]	validation_0-rmse:0.00295
[5]	validation_0-rmse:0.00273
[6]	validation_0-rmse:0.00256
[7]	validation_0-rmse:0.00246
[8]	validation_0-rmse:0.00238
[9]	validation_0-rmse:0.00236
[10]	validation_0-rmse:0.00235
[11]	validation_0-rmse:0.00234
[12]	validation_0-rmse:0.00233
[13]	validation_0-rmse:0.00232
[14]	validation_0-rmse:0.00231
[15]	validation_0-rmse:0.00229
[16]	validation_0-rmse:0.00226
[17]	validation_0-rmse:0.00226
[18]	validation_0-rmse:0.00227
[19]	validation_0-rmse:0.00227
[20]	validation_0-rmse:0.00227
[21]	validation_0-rmse:0.00225
[22]	validation_0-rmse:0.00224
[23]	validation_0-rmse:0.00223
[24]	validation_0-rmse:0.00223
[25]	validation_0-rmse:0.00223
[26]	validation_0-rmse:0.00223
[27]	validation_0-rmse:0.00222
[28]	validation_0-rmse:0.00221
[29]	validation_0-rmse:0.00222
[30]	validation_0-rmse:0.00221
[31]	validation_0-rmse:0.00221
[32]	validation_0-

In [22]:
# Put X into GPU
X_train_cp = cp.array(X_train.to_numpy().astype('float32'))
X_test_cp = cp.array(X_test.to_numpy().astype('float32'))
y_train_cp = cp.array(y_train.to_numpy().astype('float32'))
y_test_cp = cp.array(y_test.to_numpy().astype('float32'))
# # Put reg to GPU
# reg.set_params(device="cuda")
# # No warning, both on GPU
# reg.predict(X)

In [23]:
X_train_cp[0:5]

array([[7.0038214e-03, 7.0038214e-03, 0.0000000e+00, ..., 0.0000000e+00,
        0.0000000e+00, 0.0000000e+00],
       [2.8025478e-03, 2.8025478e-03, 5.8189165e-03, ..., 0.0000000e+00,
        0.0000000e+00, 0.0000000e+00],
       [2.0356688e-03, 1.9617579e-03, 5.0175092e-03, ..., 0.0000000e+00,
        0.0000000e+00, 0.0000000e+00],
       [9.0090446e-03, 9.2383437e-03, 6.0976669e-04, ..., 0.0000000e+00,
        0.0000000e+00, 0.0000000e+00],
       [5.6050955e-03, 5.0700638e-03, 0.0000000e+00, ..., 0.0000000e+00,
        1.0000000e+00, 0.0000000e+00]], dtype=float32)

In [34]:
import xgboost as xgb
from hyperopt import fmin, tpe, hp, STATUS_OK

# Define the hyperparameter space
space = {
    # 'max_depth': hp.quniform('max_depth', 2, 8, 1),
    'n_estimators': hp.choice('n_estimators', np.arange(10, 80, dtype=int)),
    'max_depth': hp.choice('max_depth', np.arange(1, 14, dtype=int)),
    'learning_rate': hp.loguniform('learning_rate', -5, -2),
    'subsample': hp.uniform('subsample', 0.5, 1)
}

# Define the objective function to minimize
def objective(params):
    xgb_model = xgb.XGBRegressor(device = 'cuda',objective="reg:squarederror", **params)
    # xgb_model = xgb.XGBClassifier(**params)
    xgb_model.fit(X_train_cp, y_train_cp)
    y_pred = xgb_model.predict(X_test_cp)
    score = mean_squared_error(y_test_cp.get(), y_pred)
    return {'loss': score, 'status': STATUS_OK}

# Perform the optimization
best_params = fmin(objective, space, algo=tpe.suggest, max_evals=100)
print("Best set of hyperparameters: ", best_params)

100%|████| 100/100 [02:08<00:00,  1.28s/trial, best loss: 4.302419256418943e-06]
Best set of hyperparameters:  {'learning_rate': 0.1008539473576813, 'max_depth': 7, 'n_estimators': 61, 'subsample': 0.5926322657761525}


In [570]:
mse=mean_squared_error(y_test, y_pred)
print(np.sqrt(mse),mse)

0.0021397579 4.5785637e-06


In [571]:
y_test_raw = dp.numeric_target_normalizer.inverse_transform(y_test.to_numpy().reshape(-1, 1))
y_pred_raw = dp.numeric_target_normalizer.inverse_transform(y_pred.reshape(-1, 1))

In [572]:
mse=mean_squared_error(y_test_raw, y_pred_raw)
print(np.sqrt(mse))

77031.28


In [573]:
feature_vals = xgb_model.feature_importances_
feature_names = xgb_model.feature_names_in_
df_from_arr = pd.DataFrame(data=[feature_names,feature_vals]).T
df_from_arr.sort_values(by=1,ascending=False).head(30)

,0,1
1,list_price,0.285642
327,Steel Siding,0.046497
350,Condomrd_tpc_,0.043798
158,Beach,0.041721
169,Fence-Invisible Pet,0.038334
0,original_list_price,0.030607
274,Gas,0.029863
177,Lake Access,0.023363
38,high_school_district_4,0.021923
179,Landscaped,0.017475


In [622]:
import numpy as np
from catboost import Pool, CatBoostRegressor
# initialize data
train_data = X_train
train_label = y_train
test_data = X_train
# initialize Pool
train_pool = Pool(train_data, 
                  train_label)
test_pool = Pool(test_data) 

# specify the training parameters 
model = CatBoostRegressor(iterations=100, 
                          depth=15, 
                          learning_rate=.01, 
                          loss_function='RMSE')
#train the model
model.fit(train_pool)
# make the prediction using the resulting model
preds = model.predict(test_pool)

0:	learn: 0.0093422	total: 2.69s	remaining: 4m 26s
1:	learn: 0.0092625	total: 4.82s	remaining: 3m 56s
2:	learn: 0.0091860	total: 7.08s	remaining: 3m 49s
3:	learn: 0.0091071	total: 9.36s	remaining: 3m 44s
4:	learn: 0.0090297	total: 12.1s	remaining: 3m 49s
5:	learn: 0.0089529	total: 14.9s	remaining: 3m 52s
6:	learn: 0.0088756	total: 17.8s	remaining: 3m 56s
7:	learn: 0.0087993	total: 20.3s	remaining: 3m 53s
8:	learn: 0.0087249	total: 20.8s	remaining: 3m 30s
9:	learn: 0.0086506	total: 23.2s	remaining: 3m 28s
10:	learn: 0.0085784	total: 26s	remaining: 3m 30s
11:	learn: 0.0085064	total: 28.8s	remaining: 3m 30s
12:	learn: 0.0084345	total: 31.9s	remaining: 3m 33s
13:	learn: 0.0083635	total: 34.8s	remaining: 3m 34s
14:	learn: 0.0082933	total: 37.5s	remaining: 3m 32s
15:	learn: 0.0082241	total: 40.1s	remaining: 3m 30s
16:	learn: 0.0081551	total: 42.7s	remaining: 3m 28s
17:	learn: 0.0080874	total: 45.3s	remaining: 3m 26s
18:	learn: 0.0080198	total: 47.8s	remaining: 3m 23s
19:	learn: 0.0079544	tot

In [52]:

## find missing school districts by zip code ##
before = df.elementary_school_district.isna().sum(),df.middle_or_junior_school_district.isna().sum(),df.high_school_district.isna().sum()

schools = ['elementary_school_district','middle_or_junior_school_district','high_school_district']
school_list = []

for dist in schools:
    df[dist] = df[dist].apply(lambda x: x.lower() if pd.notnull(x) else x)
    
for dist in schools:
    df_school = df[['postal_code',dist]].copy()
    df_school = df_school.dropna(subset=[dist],how='all')
    
    df_school = df_school.groupby(["postal_code",dist]).size().reset_index().rename(columns={0:'count'})
    school_dict = dict(zip(df_school.postal_code, df_school[dist]))

    df[dist] = df[dist].fillna(df.postal_code.map(school_dict))

after = df.elementary_school_district.isna().sum(),df.middle_or_junior_school_district.isna().sum(),df.high_school_district.isna().sum()
print(f'Reduced district nulls from {before} to {after} after.')
df[schools] = df[schools].fillna(0)

## binarize school districts transform ##
binarize_dict = {}
for dist in schools:
    df_dist = df[dist].copy()
    encoder = BinaryEncoder(cols=[dist])
    df_binarize = encoder.fit_transform(df_dist)
    df = pd.concat([df,df_binarize],axis=1)
    df = df.drop(dist,axis=1)
    binarize_dict[dist] = encoder


Reduced district nulls from (215, 220, 227) to (114, 115, 115) after.


In [54]:
binarize_dict['elementary_school_district'].feature_names_out_

['elementary_school_district_0',
 'elementary_school_district_1',
 'elementary_school_district_2',
 'elementary_school_district_3',
 'elementary_school_district_4',
 'elementary_school_district_5',
 'elementary_school_district_6',
 'elementary_school_district_7',
 'elementary_school_district_8']

In [9]:
df.describe()

,original_list_price,list_price,close_price,association_fee,tax_annual_amount,days_on_market,cumulative_days_on_market,previous_list_price,rooms_total,bedrooms_total,bathrooms_full,bathrooms_half,garage_spaces,lot_size_acres,living_area,year_built,postal_code,mrd_rehab_year,mrd_tnu,elementary_school_district_0,elementary_school_district_1,elementary_school_district_2,elementary_school_district_3,elementary_school_district_4,elementary_school_district_5,elementary_school_district_6,elementary_school_district_7,elementary_school_district_8,middle_or_junior_school_district_0,middle_or_junior_school_district_1,middle_or_junior_school_district_2,middle_or_junior_school_district_3,middle_or_junior_school_district_4,middle_or_junior_school_district_5,middle_or_junior_school_district_6,middle_or_junior_school_district_7,middle_or_junior_school_district_8,high_school_district_0,high_school_district_1,high_school_district_2,high_school_district_3,high_school_district_4,high_school_district_5,high_school_district_6,high_school_district_7
count,1.000000e+04,1.000000e+04,1.000000e+04,10000.000000,10000.000000,10000.000000,10000.000000,1.000000e+04,10000.000000,10000.000000,10000.000000,10000.000000,10000.000000,10000.000000,10000.000000,10000.000000,10000.000000,1940.000000,3389.000000,10000.000000,10000.000000,10000.000000,10000.000000,10000.000000,10000.000000,10000.000000,10000.000000,10000.000000,10000.00000,10000.000000,10000.000000,10000.000000,10000.00000,10000.000000,10000.000000,10000.000000,10000.000000,10000.00000,10000.000000,10000.000000,10000.00000,10000.000000,10000.000000,10000.000000,10000.000000
mean,3.943180e+05,3.811469e+05,3.679937e+05,183.018500,6537.443457,104.342800,144.359700,1.841316e+05,7.289400,3.061600,2.048600,0.492100,1.805290,0.478579,1923.381900,1852.539600,60474.171500,2001.432474,75.557982,0.001500,0.142800,0.241700,0.373500,0.384500,0.371900,0.430700,0.611700,0.390800,0.00180,0.144900,0.239700,0.375500,0.38130,0.374600,0.440200,0.607700,0.365300,0.02050,0.159200,0.326900,0.37610,0.374600,0.390400,0.621800,0.486800
std,4.573609e+05,4.455320e+05,4.096915e+05,364.872149,7225.207706,113.699808,184.981095,2.984740e+05,2.471368,1.067202,0.954531,0.575995,10.082226,6.811361,1371.994005,475.019951,745.745386,163.985075,151.459981,0.038703,0.349886,0.428135,0.483757,0.486501,0.483336,0.495199,0.487388,0.487954,0.04239,0.352018,0.426922,0.484276,0.48573,0.484044,0.496436,0.488287,0.481538,0.14171,0.365881,0.469104,0.48443,0.484044,0.487864,0.484962,0.499851
min,0.000000e+00,0.000000e+00,3.000000e+03,0.000000,0.000000,-52.000000,0.000000,0.000000e+00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,46311.000000,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000
25%,1.780000e+05,1.699000e+05,1.664500e+05,0.000000,2419.910000,22.000000,32.000000,0.000000e+00,5.000000,2.000000,1.000000,0.000000,1.000000,0.000000,1127.000000,1950.000000,60124.000000,2013.000000,4.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000
50%,2.847475e+05,2.750000e+05,2.700000e+05,0.000000,4929.500000,79.000000,98.000000,6.490000e+04,7.000000,3.000000,2.000000,0.000000,2.000000,0.100000,1663.000000,1974.000000,60473.000000,2018.000000,10.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,1.000000,0.000000,0.00000,0.000000,0.000000,0.00000,0.000000,0.000000,1.000000,0.000000
75%,4.490000e+05,4.299000e+05,4.220000e+05,259.000000,8484.305000,148.000000,180.000000,2.890000e+05,9.000000,4.000000,2.000000,1.000000,2.000000,0.238800,2428.000000,2001.000000,

In [10]:
## building age transform ##
df['age'] = df['year_built'].apply(lambda x: 2024-int(x) if int(x) > 0 else x)
bins = [-1, 0, 10, 20, 30, 40, 50, 60, 70, 80, 1000]
labels = ['0','1-10', '10-20', '20-30', '30-40', '40-50', '50-60', '60-70', '70-80', '80+']
df['age_label'] = pd.cut(df.age, bins=bins, labels=labels)

In [11]:
 ## boolean fields transform ##
df['waterfront_yn'] = df['waterfront_yn'].fillna(False)
df['new_construction_yn'] = df['new_construction_yn'].fillna(False)
df['mrd_disability_access'] = df['mrd_disability_access'].fillna('No')
df['mrd_disability_access'] = pd.Series(np.where(df.mrd_disability_access.values == 'Yes', True, False), df.index)
df['mrd_garage_onsite'] = df['mrd_garage_onsite'].fillna('No')
df['mrd_garage_onsite'] = pd.Series(np.where(df.mrd_garage_onsite.values == 'Yes', True, False), df.index)
df['mrd_rehab_year'] = df['mrd_rehab_year'].apply(lambda x: True if pd.notnull(x) else False)

In [1046]:
## postal code to long lat coordinates ##
import pgeocode
geolocator = pgeocode.Nominatim('US')
geocode_dict = {}
for postal_code in list(df.postal_code.unique()):
    location = geolocator.query_postal_code(str(postal_code))
    geocode_dict[postal_code] = (location.longitude,location.latitude)

df['postal_long'] = df['postal_code'].apply(lambda x: geocode_dict[x][0] if ((pd.notnull(x)) and (x in geocode_dict)) else np.nan)
df['postal_lat'] = df['postal_code'].apply(lambda x: geocode_dict[x][1] if ((pd.notnull(x)) and (x in geocode_dict)) else np.nan)

In [20]:
feature_list = ['accessibility_features',
                'heating',
                'water_source',
                'sewer',
                'lot_features',
                'roof',
                'community_features',
                'laundry_features',
                'cooling',
                'association_fee_includes',
                'mrd_din',
                'mrd_ext',
                'mrd_fireplace_location',
                'ownership',
                'mrd_bas',
                'mrd_pkn',
                'mrd_tpc',
                'mrd_tpe']
for feature in feature_list:
    df[feature] = df[feature].fillna(f'None_{feature}')
    curr_len=(len(df[feature].str.split(',\s*').explode().unique().tolist()))
    print(f'{feature} has {curr_len} unique categories.')
    df[feature] = df[feature].str.replace('[{}"]','',regex=True).str.split(',\s*')

accessibility_features has 175 unique categories.
heating has 76 unique categories.
water_source has 22 unique categories.
sewer has 25 unique categories.
lot_features has 194 unique categories.
roof has 26 unique categories.
community_features has 64 unique categories.
laundry_features has 44 unique categories.
cooling has 49 unique categories.
association_fee_includes has 80 unique categories.
mrd_din has 7 unique categories.
mrd_ext has 32 unique categories.
mrd_fireplace_location has 16 unique categories.
ownership has 6 unique categories.
mrd_bas has 25 unique categories.
mrd_pkn has 4 unique categories.
mrd_tpc has 30 unique categories.
mrd_tpe has 18 unique categories.


In [1048]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 350000 entries, 0 to 349999
Data columns (total 75 columns):
 #   Column                              Non-Null Count   Dtype   
---  ------                              --------------   -----   
 0   listing_id                          350000 non-null  object  
 1   list_office_mls_id                  350000 non-null  object  
 2   original_list_price                 350000 non-null  float64 
 3   list_price                          350000 non-null  float64 
 4   close_price                         350000 non-null  float64 
 5   association_fee                     350000 non-null  float64 
 6   tax_annual_amount                   350000 non-null  float64 
 7   days_on_market                      350000 non-null  int64   
 8   cumulative_days_on_market           350000 non-null  int64   
 9   previous_list_price                 350000 non-null  int64   
 10  rooms_total                         350000 non-null  int64   
 11  bedrooms_tota

In [21]:
from sklearn.preprocessing import MultiLabelBinarizer

feature_bin_dict = {}
for feature in feature_list:
    mlb = MultiLabelBinarizer()
    df = df.join(
            pd.DataFrame(
                    mlb.fit_transform(df.pop(feature)),
                    index=df.index,
                    columns=mlb.classes_), lsuffix=f'{feature}_')
    feature_bin_dict[feature] = mlb

In [44]:
feature_list = ['accessibility_features',
                'heating',
                'water_source',
                'sewer',
                'lot_features',
                'roof',
                'community_features',
                'laundry_features',
                'cooling',
                'association_fee_includes',
                'mrd_din',
                'mrd_ext',
                'mrd_fireplace_location',
                'ownership',
                'mrd_bas',
                'mrd_pkn',
                'mrd_tpc',
                'mrd_tpe']
list(feature_bin_dict['accessibility_features'].classes_)

['Accessible Bedroom',
 'Accessible Central Living Area',
 'Accessible Closets',
 'Accessible Common Area',
 'Accessible Doors',
 'Accessible Elevator Installed',
 'Accessible Entrance',
 'Accessible Full Bath',
 'Accessible Hallway(s)',
 'Accessible Kitchen',
 'Accessible Kitchen Appliances',
 'Accessible Stairway',
 'Accessible Washer/Dryer',
 'Adaptable Bathroom Walls',
 'Adaptable For Elevator',
 'Bath Grab Bars',
 'Ceiling Track',
 'Central Living Area',
 'Common Area',
 'Customized Wheelchair Accessible',
 'Disabled Parking',
 'Door Width 32 Inches or More',
 'Enhanced Accessible',
 'Entry Slope less than 1 foot',
 'Exterior Wheelchair Lift',
 'Flashing Doorbell',
 'Flooring Modifications',
 'Grab Bars Throughout',
 'Grip-Accessible Features',
 'Hall Width 36 Inches or More',
 'Hearing Modifications',
 'Kitchen Modifications',
 'Lever Door Handles',
 'Low Bathroom Mirrors',
 'Low Closet Rods',
 'Low Pile Carpeting',
 'Lowered Light Switches',
 'Main Level Entry',
 'Modified Wall 

In [1051]:
df.garage_spaces.describe()

count    350000.000000
mean          1.752499
std           7.831814
min           0.000000
25%           1.000000
50%           2.000000
75%           2.000000
max         999.000000
Name: garage_spaces, dtype: float64

In [1052]:
## rooms binning transform ##
rooms_features = ['rooms_total','bedrooms_total','bathrooms_full','bathrooms_half','garage_spaces']
for feature in rooms_features:
    min_val = -1
    zero_val = 0
    first_quant = 1
    second_quant = 2
    third_quant = 3
    last_quant = 4
    extra_quant = 5
    max_val = 1000
    bins = [min_val,zero_val,first_quant,second_quant,third_quant,last_quant,extra_quant,max_val]
    labels = [f'{zero_val}',f'{first_quant}',f'{second_quant}',f'{third_quant}',f'{last_quant}',f'{extra_quant}',f'{extra_quant}+']
    df[feature] = pd.cut(df[feature], bins=bins, labels=labels, duplicates='drop')

In [1053]:
df[rooms_features].head()

,rooms_total,bedrooms_total,bathrooms_full,bathrooms_half,garage_spaces
0,5+,3,1,0,2
1,5+,3,2,1,1
2,5+,3,3,1,1
3,4,1,1,1,1
4,5+,2,2,0,1


In [1054]:
df.mrd_tnu = df.mrd_tnu.fillna(0)
df.postal_long = df.postal_long.fillna(0)
df.postal_lat = df.postal_lat.fillna(0)

In [1055]:
df.info(verbose=True,show_counts=True)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 350000 entries, 0 to 349999
Data columns (total 444 columns):
 #    Column                                            Non-Null Count   Dtype   
---   ------                                            --------------   -----   
 0    listing_id                                        350000 non-null  object  
 1    list_office_mls_id                                350000 non-null  object  
 2    original_list_price                               350000 non-null  float64 
 3    list_price                                        350000 non-null  float64 
 4    close_price                                       350000 non-null  float64 
 5    association_fee                                   350000 non-null  float64 
 6    tax_annual_amount                                 350000 non-null  float64 
 7    days_on_market                                    350000 non-null  int64   
 8    cumulative_days_on_market                         350000 non-n

In [1057]:
boolean = ['elementary_school_district_0',
            'elementary_school_district_1',
            'elementary_school_district_2',
            'elementary_school_district_3',
            'elementary_school_district_4',
            'elementary_school_district_5',
            'elementary_school_district_6',
            'elementary_school_district_7',
            'elementary_school_district_8',
            'middle_or_junior_school_district_0',
            'middle_or_junior_school_district_1',
            'middle_or_junior_school_district_2',
            'middle_or_junior_school_district_3',
            'middle_or_junior_school_district_4',
            'middle_or_junior_school_district_5',
            'middle_or_junior_school_district_6',
            'middle_or_junior_school_district_7',
            'middle_or_junior_school_district_8',
            'high_school_district_0',
            'high_school_district_1',
            'high_school_district_2',
            'high_school_district_3',
            'high_school_district_4',
            'high_school_district_5',
            'high_school_district_6',
            'high_school_district_7',
            'Accessible Bedroom',
            'Accessible Central Living Area',
            'Accessible Closets',
            'Accessible Common Area',
            'Accessible Doors',
            'Accessible Elevator Installed',
            'Accessible Entrance',
            'Accessible Full Bath',
            'Accessible Hallway(s)',
            'Accessible Kitchen',
            'Accessible Washer/Dryer',
            'Adaptable For Elevator',
            'Bath Grab Bars',
            'Central Living Area',
            'Disabled Parking',
            'Door Width 32 Inches or More',
            'Entry Slope less than 1 foot',
            'Flashing Doorbell',
            'Flooring Modifications',
            'Grab Bars Throughout',
            'Grip-Accessible Features',
            'Hall Width 36 Inches or More',
            'Hearing Modifications',
            'Kitchen Modifications',
            'Lever Door Handles',
            'Low Bathroom Mirrors',
            'Low Closet Rods',
            'Low Pile Carpeting',
            'Lower Fixtures',
            'Lowered Light Switches',
            'Main Level Entry',
            'No Interior Steps',
            'None_accessibility_features',
            'Otherheating_',
            'Other Main Level Modifications',
            'Pocket Door(s)',
            'Ramp - Main Level',
            'Roll-In Shower',
            'Stair Lift',
            'Swing In Door(s)',
            'Thresholds less than 5/8 of an inch',
            'Two or More Access Exits',
            'Vehicle Transfer Area',
            'Visitable',
            'Visitor Bathroom',
            'Wheelchair Accessible',
            'Wheelchair Adaptable',
            'Wheelchair Height Mailbox',
            'Wheelchair Height Shelves',
            'Wheelchair Modifications',
            'Wheelchair Ramp(s)',
            'Baseboard',
            'Electriccooling_',
            'Forced Air',
            'Geothermalcooling_',
            'Gravity Air',
            'Heat Pump',
            'Indv Controls',
            'Natural Gas',
            'Nonelot_features_',
            'Oil',
            'Otherwater_source_',
            'Propane',
            'Radiant',
            'Radiator(s)',
            'Sep Heating Systems - 2+',
            'Solar',
            'Space Heater',
            'Steam',
            'Wood',
            'Zonedcooling_',
            'Community Well',
            'Company Well',
            'Lake Michigan',
            'None_water_source',
            'Othersewer_',
            'Private',
            'Private Well',
            'Public',
            'Shared Well',
            'Holding Tank',
            'None_sewer',
            'Otherlot_features_',
            'Overhead Sewers',
            'Public Sewer',
            'Septic Shared',
            'Septic-Mechanical',
            'Septic-Private',
            'Sewer-Storm',
            'Adjoins Government Land',
            'Backs to Open Grnd',
            'Backs to Public GRND',
            'Backs to Trees/Woods',
            'Beach',
            'Chain Link Fence',
            'Chain of Lakes Frontage',
            'Channel Front',
            'Common Grounds',
            'Corner Lot',
            'Creek',
            'Cul-De-Sac',
            'Dimensions to Center of Road',
            'Dockcommunity_features_',
            'Electric Fence',
            'Fence-Invisible Pet',
            'Fenced Yard',
            'Forest Preserve Adjacent',
            'Garden',
            'Golf Course Lot',
            'Horses Allowed',
            'Irregular Lot',
            'Lake Access',
            'Lake Front',
            'Landscaped',
            'Legal Non-Conforming',
            'Level',
            'Mature Trees',
            'Nature Preserve Adjacent',
            'Nonelaundry_features_',
            'None_lot_features',
            'Otherroof_',
            'Outdoor Lighting',
            'Paddock',
            'Park Adjacent',
            'Partial Fencing',
            'Pasture',
            'Pie Shaped Lot',
            'Pond(s)',
            'Rear of Lot',
            'River Front',
            'Sidewalkscommunity_features_',
            'Sloped',
            'Spring(s)',
            'Stream(s)',
            'Streetlights',
            'Views',
            'Water Garden',
            'Water Rightscommunity_features_',
            'Water View',
            'Waterfront',
            'Wetlands adjacent',
            'Wood Fence',
            'Wooded',
            'Woven Wire Fence',
            'Asphalt',
            'Metal',
            'None_roof',
            'Othercommunity_features_',
            'Rubber',
            'Shake',
            'Slatemrd_ext_',
            'Tar and Gravel',
            'Tile',
            'Airport/Runway',
            'Clubhouseassociation_fee_includes_',
            'Curbs',
            'Dock',
            'Gated',
            'Horse-Riding Area',
            'Horse-Riding Trails',
            'Lake',
            'None_community_features',
            'Othercooling_',
            'Park',
            'Poolassociation_fee_includes_',
            'Sidewalks',
            'Stable(s)',
            'Street Lights',
            'Street Paved',
            'Tennis Court(s)',
            'Water Rights',
            'Common Area',
            'Electric Dryer Hookup',
            'Gas Dryer Hookup',
            'In Bathroom',
            'In Garage',
            'In Kitchen',
            'In Unit',
            'Laundry Chute',
            'Laundry Closet',
            'Multiple Locations',
            'Nonecooling_',
            'None_laundry_features',
            'Sink',
            'Central Air',
            'Dual',
            'Electric',
            'Gasassociation_fee_includes_',
            'Geothermal',
            'High Efficiency (SEER 14+)',
            'Noneassociation_fee_includes_',
            'None_cooling',
            'Otherassociation_fee_includes_',
            'Partial',
            'Space Pac',
            'Window/Wall Unit - 1',
            'Window/Wall Units - 2',
            'Window/Wall Units - 3+',
            'Zoned',
            'Air Conditioning',
            'Clubhouse',
            'Doorman',
            'Electricity',
            'Exercise Facilities',
            'Exterior Maintenance',
            'Gas',
            'Heat',
            'Insurance',
            'Internet',
            'Lake Rights',
            'Lawn Care',
            'Nonemrd_din_',
            'None_association_fee_includes',
            'Othermrd_ext_',
            'Parking',
            'Pool',
            'Scavenger',
            'Security',
            'Snow Removal',
            'TV/Cable',
            'Taxes',
            'Water',
            'Combined w/ FamRm',
            'Combined w/ LivRm',
            'Kitchen/Dining Combo',
            'L-shaped',
            'Nonemrd_bas_',
            'None_mrd_din',
            'Separate',
            'Aluminum Siding',
            'Asbestos Siding',
            'Block',
            'Brick',
            'Brick Veneer',
            'Brick/Stone Msn Pred',
            'Brk/Stn Veneer Frnt',
            'Cedar',
            'Clad Trim',
            'Combination',
            'Concrete',
            'EIFS (e.g. Dryvit).',
            'Fiber Cement',
            'Fl Brick/Stn Veneer',
            'Frame',
            'Glass',
            'Limestone',
            'Log',
            'Marble/Granite',
            'Masonite',
            'None_mrd_ext',
            'Othermrd_fireplace_location_',
            'Shakes',
            'Shingle Siding',
            'Slate',
            'Steel Siding',
            'Stone',
            'Stucco',
            'Vinyl Siding',
            'Wood Siding',
            'Basement',
            'Bedroom',
            'Den/Library',
            'Dining Room',
            'Exterior',
            'Family Room',
            'Great Room',
            'Hearth Room',
            'Kitchen',
            'Living Room',
            'Loft',
            'Master Bedroom',
            'None_mrd_fireplace_location',
            'Othermrd_bas_',
            'Co-op',
            'Condomrd_tpc_',
            'Fee Simple',
            'Fee Simple w/ HO Assn.',
            'Leasehold',
            'None_ownership',
            '8 ft + pour',
            '9 ft + pour',
            'Bathroom Rough-In',
            'Cellar',
            'Concrete (Basement)',
            'Concrete Block',
            'Crawl',
            'Egress Window',
            'Exterior Access',
            'Finished',
            'Lookout',
            'None',
            'None_mrd_bas',
            'Othermrd_tpc_',
            'Partially Finished',
            'Rec/Family Area',
            'Roughed-In Fireplace',
            'Slab',
            'Sleeping Area',
            'Stone/Rock',
            'Storage Space',
            'Sub-Basement',
            'Unfinished',
            'Walk-Up Access',
            'Garage',
            'None_mrd_pkn',
            'Space/s',
            '1/2 Duplex',
            'Cluster',
            'Condo',
            'Condo-Duplex',
            'Condo-Loft',
            'Courtyard',
            'Flat',
            'Garden Complex',
            'Garden Unit',
            'Ground Level Ranch',
            'High Rise (7+ Stories)',
            'Low Rise (1-3 Stories)',
            'Manor Home/Coach House/Villa',
            'Mid Rise (4-6 Stories)',
            'None_mrd_tpc',
            'Othermrd_tpe_',
            'Penthouse',
            'Quad-2 Story',
            'Quad-Ranch',
            'Quad-Split Level',
            'Split Levelmrd_tpe_',
            'Studio',
            'T3-Townhouse 3+ Stories',
            'Townhouse-2 Story',
            'Townhouse-Ranch',
            'Townhouse-TriLevel',
            'Vintage',
            '1 Story',
            '1.5 Story',
            '2 Stories',
            '2.5 Story',
            '3 Stories',
            '4+ Stories',
            'Coach House',
            'Earth',
            'Hillside',
            'Manufactured',
            'Multi-Level',
            'None_mrd_tpe',
            'Other',
            'Raised Ranch',
            'Split Level',
            'Split Level w/ Sub',
            'Tear Down',
            'mrd_garage_onsite',
            'new_construction_yn',  
            'waterfront_yn',
            'mrd_disability_access']
numerical = ['association_fee',
            'tax_annual_amount',
            'days_on_market',
            'cumulative_days_on_market',
            'previous_list_price',
            'postal_long',
            'postal_lat',
            'living_area',
            'mrd_tnu',
            'original_list_price',
            'list_price']
categorical = ['age_label',
                'rooms_total',
                'bedrooms_total',
                'bathrooms_full',
                'bathrooms_half',
                'garage_spaces']

In [1058]:
scaler = MinMaxScaler()
df[numerical] = scaler.fit_transform(df[numerical])

In [1059]:
target_scaler = MinMaxScaler()
df.close_price = target_scaler.fit_transform(df.close_price.to_numpy().reshape(-1, 1))

In [1060]:
from sklearn.preprocessing import OneHotEncoder

ohe = OneHotEncoder()

for cat in categorical:
    df_ohe = df[cat].values.reshape(-1, 1)
    enc = OneHotEncoder().fit(df_ohe)
    df_ohe = enc.transform(df_ohe).toarray()
    df_ohe = pd.DataFrame(df_ohe, columns=enc.categories_)
    df_ohe = df_ohe.add_prefix(f'{cat}_')
    df = pd.concat([df,df_ohe], axis=1)
    df = df.drop(cat,axis=1)

In [1061]:
df.head()

,listing_id,list_office_mls_id,original_list_price,list_price,close_price,association_fee,tax_annual_amount,days_on_market,cumulative_days_on_market,previous_list_price,lot_size_acres,living_area,year_built,mrd_type,postal_code,waterfront_yn,mrd_disability_access,mrd_garage_onsite,new_construction_yn,mrd_rehab_year,mrd_tnu,elementary_school_district_0,elementary_school_district_1,elementary_school_district_2,elementary_school_district_3,elementary_school_district_4,elementary_school_district_5,elementary_school_district_6,elementary_school_district_7,elementary_school_district_8,middle_or_junior_school_district_0,middle_or_junior_school_district_1,middle_or_junior_school_district_2,middle_or_junior_school_district_3,middle_or_junior_school_district_4,middle_or_junior_school_district_5,middle_or_junior_school_district_6,middle_or_junior_school_district_7,middle_or_junior_school_district_8,high_school_district_0,high_school_district_1,high_school_district_2,high_school_district_3,high_school_district_4,high_school_district_5,high_school_district_6,high_school_district_7,high_school_district_8,age,postal_long,postal_lat,Accessible Bedroom,Accessible Central Living Area,Accessible Closets,Accessible Common Area,Accessible Doors,Accessible Electrical and Environmental Controls,Accessible Elevator Installed,Accessible Entrance,Accessible Full Bath,Accessible Hallway(s),Accessible Kitchen,Accessible Kitchen Appliances,Accessible Stairway,Accessible Washer/Dryer,Accessible for Hearing-Impairment,Adaptable Bathroom Walls,Adaptable For Elevator,Bath Grab Bars,Ceiling Track,Central Living Area,Common Arealaundry_features_,Customized Wheelchair Accessible,Disabled Parking,Door Width 32 Inches or More,Electronic Environmental Controls,Enhanced Accessible,Entry Slope less than 1 foot,Exterior Wheelchair Lift,Flashing Doorbell,Flooring Modifications,Grab Bars Throughout,Grip-Accessible Features,Hall Width 36 Inches or More,Hearing Modifications,Kitchen Modifications,Lever Door Handles,Low Bathroom Mirrors,Low Closet Rods,Low Pile Carpeting,Lower Fixtures,Lowered Light Switches,Main Level Entry,Modified Wall Outlets,No Interior Steps,None_accessibility_features,Otherheating_,Other Main Level Modifications,Pocket Door(s),Ramp - Main Level,Reinforced Floors,Roll-In Shower,Safe Emergency Egress from Home,Smart Technology,Stair Lift,Standby Generator,Swing In Door(s),Therapeutic Whirlpool,Thresholds less than 5/8 of an inch,Two or More Access Exits,Vehicle Transfer Area,Visitable,Visitor Bathroom,Walker-Accessible Stairs,Wheelchair Accessible,Wheelchair Adaptable,Wheelchair Height Mailbox,Wheelchair Height Shelves,Wheelchair Modifications,Wheelchair Ramp(s),Baseboard,Electriccooling_,Floor Furnace,Forced Air,Geothermalcooling_,Gravity Air,Heat Pump,Indv Controls,Natural Gas,Nonelot_features_,None_heating,Oil,Otherwater_source_,Propane,Radiant,Radiator(s),Sep Heating Systems - 2+,Solar,Space Heater,Steam,Wood,Zonedcooling_,Community Well,Company Well,Lake Michigan,None_water_source,Othersewer_,Private,Private Well,Public,Shared Well,Holding Tank,None_sewer,Otherlot_features_,Overhead Sewers,Public Sewer,Septic Shared,Septic-Mechanical,Septic-Private,Sewer-Storm,Adjoins Government Land,Backs to Open Grnd,Backs to Public GRND,Backs to Trees/Woods,Beach,Chain Link Fence,Chain of Lakes Frontage,Channel Front,Common Grounds,Corner Lot,Creek,Cul-De-Sac,Dimensions to Center of Road,Dockcommunity_features_,Electric Fence,Fence-Invisible Pet,Fenced Yard,Forest Preserve Adjacent,Garden,Golf Course Lot,Horses Allowed,Infill Lot,Irregular Lot,Lake Access,Lake Front,Landscaped,Legal Non-Conforming,Level,Mature Trees,Nature Preserve Adjacent,Nonelaundry_features_,None_lot_features,Otherroof_,Outdoor Lighting,Paddock,Park Adjacent,Partial Fencing,Pasture,Pie Shaped Lot,Pond(s),Rear of Lot,River Front,Sidewalkscommunity_features_,Sloped,Spring(s),Stream(s),Streetlights,Views,Water Garden,Water Rightscommunity_features_,Water View,Waterfront,Wetlands adjacent,Wood

In [1062]:
df.info(verbose=True,show_counts=True)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 350000 entries, 0 to 349999
Data columns (total 485 columns):
 #    Column                                            Non-Null Count   Dtype  
---   ------                                            --------------   -----  
 0    listing_id                                        350000 non-null  object 
 1    list_office_mls_id                                350000 non-null  object 
 2    original_list_price                               350000 non-null  float64
 3    list_price                                        350000 non-null  float64
 4    close_price                                       350000 non-null  float64
 5    association_fee                                   350000 non-null  float64
 6    tax_annual_amount                                 350000 non-null  float64
 7    days_on_market                                    350000 non-null  float64
 8    cumulative_days_on_market                         350000 non-null  float

In [1063]:
df = df.drop(['listing_id','list_office_mls_id','mrd_type','year_built','postal_code'],axis=1)

In [1064]:
df.info(verbose=True,show_counts=True)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 350000 entries, 0 to 349999
Data columns (total 480 columns):
 #    Column                                            Non-Null Count   Dtype  
---   ------                                            --------------   -----  
 0    original_list_price                               350000 non-null  float64
 1    list_price                                        350000 non-null  float64
 2    close_price                                       350000 non-null  float64
 3    association_fee                                   350000 non-null  float64
 4    tax_annual_amount                                 350000 non-null  float64
 5    days_on_market                                    350000 non-null  float64
 6    cumulative_days_on_market                         350000 non-null  float64
 7    previous_list_price                               350000 non-null  float64
 8    lot_size_acres                                    350000 non-null  float

In [1065]:
y = df['close_price']
X = df.drop('close_price',axis=1)

In [1066]:
import xgboost as xgb
from sklearn.metrics import auc, accuracy_score, confusion_matrix, mean_squared_error
from sklearn.model_selection import cross_val_score, GridSearchCV, KFold, RandomizedSearchCV, train_test_split

xgb_model = xgb.XGBRegressor(objective="reg:squarederror", n_estimators=40, random_state=42)

X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42)

xgb_model.fit(X_train, y_train, eval_set=[(X_test, y_test)])

y_pred = xgb_model.predict(X_test)

[0]	validation_0-rmse:0.00657
[1]	validation_0-rmse:0.00487
[2]	validation_0-rmse:0.00371
[3]	validation_0-rmse:0.00300
[4]	validation_0-rmse:0.00255
[5]	validation_0-rmse:0.00230
[6]	validation_0-rmse:0.00216
[7]	validation_0-rmse:0.00209
[8]	validation_0-rmse:0.00204
[9]	validation_0-rmse:0.00201
[10]	validation_0-rmse:0.00198
[11]	validation_0-rmse:0.00198
[12]	validation_0-rmse:0.00198
[13]	validation_0-rmse:0.00197
[14]	validation_0-rmse:0.00196
[15]	validation_0-rmse:0.00196
[16]	validation_0-rmse:0.00195
[17]	validation_0-rmse:0.00194
[18]	validation_0-rmse:0.00193
[19]	validation_0-rmse:0.00192
[20]	validation_0-rmse:0.00193
[21]	validation_0-rmse:0.00192
[22]	validation_0-rmse:0.00191
[23]	validation_0-rmse:0.00191
[24]	validation_0-rmse:0.00190
[25]	validation_0-rmse:0.00190
[26]	validation_0-rmse:0.00191
[27]	validation_0-rmse:0.00192
[28]	validation_0-rmse:0.00191
[29]	validation_0-rmse:0.00191
[30]	validation_0-rmse:0.00192
[31]	validation_0-rmse:0.00192
[32]	validation_0-